# A3 - Bonus - cs2951-R
Shaun Wallace 
File: breathing.py
(Python 3.5.1 & Anaconda 4, Mac OS X 10.9.5)

##### Goal
"Lay down and place your phone on your chest.  Capture 10 minutes of normal breathing and
heart beats.  Can you write a function that takes this data to figure out your respiratory rate
(breaths per second) or heart rate?" -A3

In order to achieve this three processes needed to be developed:
1. A Low Pass Filter, to remove the noise from the data.
2. FFT functions to gain access to various data. *Such as Pitch Detection
3. A Pitch Detection and Conversion tool.


##### Low Pass Filter + FFT

It was necessary to use a windowing function in order to properly divide the data up into bins.  This will enable the filter to reduce high frequency content.  It will also allow it to remove gaussian noise:
```python
win = signal.hann(6) #LP Filter using hanning window 
yFilt = signal.convolve(y, win, mode='same') / sum(win)
```
In order to test this filter a basic complex waveform was computed using the sum of sines.
```python
y = sin(2*pi*ff*t)
y = y + .6*sin(4*pi*ff*t) + .4*sin(8*pi*ff*t) + .2*sin(12*pi*ff*t) #summing sines
```
![title](img/complexwave.png)
This demonstrates the filter is smoothing out the waveform and removing high frequency content.  The blue x's represent the original peak energy per frequency.  While we can visually ascertain the pitch is 30 Hz we need to code a basic pitch detection algorithm. Once we can extract the fundamental frequency we can use that later on to figure out Breaths per Minute.

##### Pitch Detection

```python
def pitchDetection(y,t,title):
    yfft = fft(y)
    FFT = abs(sp.fft(y))
    freqs = sp.fftpack.fftfreq(len(t), 1)
    
    #replace negative values
    FFT[abs(FFT) < 0.000001]= 0 #some zeros
    FFT = np.ma.masked_equal(FFT,0)
    
    maxPos = FFT.argmax(axis=0) #finds position in FFT with highest value = most energy
    fundamentalFreq = abs(freqs.real[maxPos]) #uses maxPos to find the corresponding fundamental freq
    print('Fundamental Frequency Sine Wave ',title,'= ', fundamentalFreq, ' Hz (Beats per Second)')
```
```
Fundamental Frequency Sine Wave  Normal =  30.0  Hz (Beats per Second)
Fundamental Frequency Sine Wave  Filtered =  30.0  Hz (Beats per Second)
```

##### Run on Breathing Data

After I completed collecting my data using AndroSensor on Android. I tookout a 10 minute section from the middle.  Instead of plotting relationships in Excel/Sheets I laid the phone on my chest and sat in a position where I could read the sensors.  I then carefull went through and looked at each sensor in real time.  What I was looking for were sensors who appeared to have nice sinusoidal waveforms on the application's display.  From this excercise I chose two:

![title](img/accXlong2.png)
![title](img/gyroYlong2.png)

The two images above are scaled form -1 to 1.  From the images above the LP Filter does an excellent job of smoothing out the peaks and reducing noise.  The data itself and the algorithm work fairly well on their own:

Scaled -1 to 1 to represent waveform: (hann.window(50))
```
Fundamental Frequency  Gyroscope Y =  [ 21.88]  Breaths per Minute (Avg Adult is 12-18)
Fundamental Frequency  Acceleration X =  [ 29.14]  Breaths per Minute (Avg Adult is 12-18)
Fundamental Frequency  Gyroscope Y - Filtered  =  [ 20.84]  Breaths per Minute (Avg Adult is 12-18)
Fundamental Frequency  Acceleration X - Filtered  =  [ 29.66]  Breaths per Minute (Avg Adult is 12-18)
```
Not Scaled Raw Data: (hann.window(50))
```
Fundamental Frequency  Gyroscope Y =  [ 21.88]  Breaths per Minute (Avg Adult is 12-18)
Fundamental Frequency  Acceleration X =  [ 20.75]  Breaths per Minute (Avg Adult is 12-18)
Fundamental Frequency  Gyroscope Y - Filtered  =  [ 20.55]  Breaths per Minute (Avg Adult is 12-18)
Fundamental Frequency  Acceleration X - Filtered  =  [ 20.69]  Breaths per Minute (Avg Adult is 12-18)
```

The data is fairly accurate.  I used a hanning window of 50 to help smooth out the rough peaks.  I kept increasing the hanning window all the way up to 800.  It got both readings down to 18 and 19 breaths per minute.  However this data was too smoothed to be taken as truth.  A really good way to further explore the data is to pull out various chunks from the 6000 rows:
![title](img/gyro_50_250.png)
hann.window(20)
```
Fundamental Frequency  Gyroscope Y =  [ 0.3]  Breaths per Minute (Avg Adult is 12-18)
Fundamental Frequency  Gyroscope Y - Filtered  =  [ 14.7]  Breaths per Minute (Avg Adult is 12-18)
```
![title](img/acc1900_2000.png)
hann.window(20)
```
Fundamental Frequency  Acceleration X =  [ 22.2]  Breaths per Minute (Avg Adult is 12-18)]
Fundamental Frequency  Acceleration X - Filtered  =  [ 17.4]  Breaths per Minute (Avg Adult is 12-18)
```

##### Final Thoughts

We can see from these two examples that the filtering can work both ways to increase the accuracy on smaller samples sizes.  These methods are not full proof.  They tend to work best with Low Frequency waveforms.  Generally anything below 45 Hz.  Filtering helps for higher waveforms but there is always a tradeoff.  Something that is nice, is that the algorithm scales as you modify your sample size.  Hence why the various graphs above worked with different sample sizes.  The general range of results for individual snippets runs from ~0.5 to 30.0 beats per minute.

*My code runs on my local machine.  There are a few general warnings due to certain data handling methods that have started to be deprecated in newer versions of Python.  If you have any questions please do not hesitate to contact me.

```python
import numpy as np
import sklearn as sk
import scipy as sp
import pylab

from sklearn import preprocessing
from numpy import sin, linspace, pi
from pylab import plot, show, title, xlabel, ylabel, subplot
from scipy import fft, arange, signal

def plotSpectrum(y,Fs, color):
    n = len(y) # length of the signal
    k = arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[1:n/2] # one side frequency range (sliced the frq in half)

    Y = fft(y)/n # fft computing and normalization
    Y = Y[1:n/2]

    plot(frq,abs(Y),color) # plotting the spectrum
    xlabel('Freq (Hz)')
    ylabel('|Y(freq)|')

def pitchDetection(y,t,title):
    FFT = abs(sp.fft(y))
    freqs = sp.fftpack.fftfreq(t.size, t[1]-t[0])

    #replace negative values
    FFT[abs(FFT) < 0.000001]= 0 # some zeros
    FFT = np.ma.masked_equal(FFT,0)
    maxPos = FFT.argmax(axis=0) #finds position in FFT with highest value = most energy
    fundamentalFreq = abs(freqs.real[maxPos]) #uses maxPos to find the corresponding fundamental freq
    print('Fundamental Frequency Sine Wave ',title,'= ', fundamentalFreq, ' Hz (Beats per Second)')


def pitchDetectionData(y,t,title):
    yfft = fft(y)

    FFT = abs(sp.fft(y))
    freqs = sp.fftpack.fftfreq(len(t), 1)

    #replace negative values
    FFT[abs(FFT) < 0.000001]= 0 # some zeros
    FFT = np.ma.masked_equal(FFT,0)

    maxPos = FFT.argmax(axis=0) #finds position in FFT with highest value = most energy
    fundamentalFreq = abs(freqs.real[maxPos]) #uses maxPos to find the corresponding fundamental freq
    print('Fundamental Frequency ',title,'= ', fundamentalFreq*60, ' Breaths per Minute (Avg Adult is 12-18)')

###START-PROOF#####################################################################
Fs = 100.0;  # sampling rate
Ts = 1.0/Fs; # sampling interval
t = arange(0,1,Ts) # time vector

ff = 30;   # frequency of the test signal
y = sin(2*pi*ff*t)
y = y + .6*sin(4*pi*ff*t) + .4*sin(8*pi*ff*t) + .2*sin(12*pi*ff*t) #summing sines

win = signal.hann(6) #LP Filter using hanning window
yFilt = signal.convolve(y, win, mode='same') / sum(win)

#compute FFT
yfft = fft(y)
FFT = abs(sp.fft(y))
freqs = sp.fftpack.fftfreq(t.size, t[1]-t[0])
#replace negative values
FFT[abs(FFT) < 0.000001] = 0 # some zeros
FFT = np.ma.masked_equal(FFT,0)


pitchDetection(y,t,'Normal')
pitchDetection(yFilt,t,'Filtered')

pylab.subplot(212)
pylab.plot(abs(freqs.real), sp.log10(FFT), 'x')

subplot(2,1,1)
plot(t,y, 'g')
plot(t,yFilt, 'r')
title('Complex Waveform - 30 Hz')
xlabel('Time')
ylabel('Amplitude')
subplot(2,1,2)
plotSpectrum(y,Fs, 'g')
plotSpectrum(yFilt,Fs, 'r')
show()

###START-DATA##################################################################
data = np.genfromtxt(('_DataRaw/Breathing.csv'), delimiter=',', skip_header=1)

scaler = preprocessing.MinMaxScaler([-1,1]) #fits better with pitch detection

mnL = 750
mxL = 950
mnL = 1000
mxL = 1400
mnL = 1900
mxL = 2300
mnL = 1900
mxL = 2000
mnL = 50
mxL = 250
time = data[mnL:mxL,18]
#accelX = scaler.fit_transform(data[mnL:mxL, 0])
#gyroY = scaler.fit_transform(data[mnL:mxL, 10])
accelX = data[mnL:mxL, 0]
gyroY = data[mnL:mxL, 10]

win = signal.hann(20)
accelXFilt = signal.convolve(accelX, win, mode='same') / sum(win)
gyroYFilt = signal.convolve(gyroY, win, mode='same') / sum(win)

time = np.array(time).reshape(len(time), 1)
accelX = np.array(accelX).reshape(len(accelX), 1)
gyroY = np.array(gyroY).reshape(len(gyroY), 1)
accelXFilt = np.array(accelXFilt).reshape(len(accelXFilt), 1)
gyroYFilt = np.array(gyroYFilt).reshape(len(gyroYFilt), 1)

pitchDetectionData(gyroY,time,'Gyroscope Y')
pitchDetectionData(accelX,time,'Acceleration X')
pitchDetectionData(gyroYFilt,time,'Gyroscope Y - Filtered ')
pitchDetectionData(accelXFilt,time,'Acceleration X - Filtered ')

title('Acceleration X ')
ylabel('Filtered (blue) & Normal (green)')
xlabel('Time')
plot(time, accelX, 'y')
plot(time, accelXFilt, 'b')
show()

title('Gyroscope Y')
plot(time, gyroY, 'y')
plot(time, gyroYFilt, 'b')
show()
```